In [1]:
!pip install natsort
import os
import shutil
from natsort import natsorted
import re

import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import glob

sns.set(style='darkgrid')

You should consider upgrading via the '/home/xin/miniconda3/bin/python -m pip install --upgrade pip' command.


# Table 1 

## M3IL(ours 70.4%)

In [63]:

data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])
print(data)

Empty DataFrame
Columns: []
Index: [25000, 50000, 75000, 100000, 125000, 150000, 175000, 200000, 225000, 250000, 275000, 300000, 325000, 350000, 375000, 400000]


In [64]:
def get_score_from_json(file):
    try:
        with open(file + '/result.json', 'r') as f:
            score = json.load(f).get('final_suc')
    except:
        score = -np.inf
    return score


In [70]:
def get_scores_from_seed_sentense(seed, sentense_type, text_type='random_instruction'):

    experiment_name = f'mill_{text_type}_{seed}_1e-07'
    result_path = f'../log_eval/{experiment_name}/eval/first_last_sentence_type_{sentense_type}/random_seed_*'
    result_dir = natsorted(glob.glob(result_path))
    result_files = natsorted(glob.glob(f'{result_dir[0]}/evaluated_gifs/*'))

    test_score = []
    train_score = []
    for file in result_files:
        # print(f"{file}/result.json")
        if file[-5:] == 'train':
            train_score.append(get_score_from_json(file))
        else:
            test_score.append(get_score_from_json(file))
    assert(len(train_score) == len(test_score) == 16)
    return train_score, test_score


def get_final_score(train_score, test_score):
    
    # get final score which is the test score at the highest train score.
    epochs = [i for i, x in enumerate(train_score) if x == max(train_score)]
    
    return np.max([test_score[i] for i in epochs])


In [71]:
sentense_type= 'image' # or image
# seed= '1' # or image
final_score_list = []
for seed in [1, 2, 3]:
    train_score, test_score = get_scores_from_seed_sentense(seed, sentense_type)
    final_score = get_final_score(train_score, test_score)
    final_score_list.append(final_score)
print(final_score_list)
print('mean', np.around(np.mean(final_score_list), 3), 'std', np.around(np.std(final_score_list), 3))

[0.722972972972973, 0.713963963963964, 0.6779279279279279]
mean 0.705 std 0.019


## PETNet(67.6%)

In [67]:
# load from numpyz.
def get_data(file):
    d = np.load(file, allow_pickle=True)
    return pd.DataFrame(d['arr_1'], columns=d['arr_0'])

In [72]:
pd_list = []
file = './ex2_numpy/petnet_image.npz'
all_result = get_data(file)
all_result['epoch'] = [str((i+1) * 25) for i in range(16)]
all_result = all_result.set_index('epoch', drop=True).round(3)
# all_result.rename(columns=lambda x: x.replace("_15types_1_1e-07", "").replace("task_id_1", "task_id").replace("new_test", "newtest"), inplace=True)
all_result

,petnet_image_test_1,petnet_image_train_1,petnet_image_new_test_1,petnet_image_test_3,petnet_image_train_3,petnet_image_new_test_3,petnet_image_test_4,petnet_image_train_4,petnet_image_new_test_4,petnet_image_test_5,...,petnet_image_new_test_5,petnet_image_test_100,petnet_image_train_100,petnet_image_new_test_100,petnet_image_test_200,petnet_image_train_200,petnet_image_new_test_200,petnet_image_test_300,petnet_image_train_300,petnet_image_new_test_300
epoch,,,,,,,,,,,,,,,,,,,,,
25,0.243,0.258,0.207,0.255,0.342,0.169,0.273,0.333,0.160,0.257,...,0.218,0.243,0.258,0.207,0.250,0.183,0.227,0.255,0.342,0.169
50,0.304,0.350,0.252,0.320,0.408,0.245,0.365,0.408,0.223,0.288,...,0.223,0.304,0.350,0.252,0.320,0.367,0.205,0.320,0.408,0.245
75,0.502,0.542,0.378,0.520,0.500,0.376,0.489,0.525,0.403,0.473,...,0.414,0.502,0.542,0.378,0.520,0.533,0.417,0.520,0.500,0.376
100,0.475,0.483,0.374,0.468,0.458,0.372,0.484,0.517,0.376,0.500,...,0.394,0.475,0.483,0.374,0.471,0.500,0.354,0.468,0.458,0.372
125,0.423,0.458,0.313,0.421,0.583,0.347,0.421,0.517,0.302,0.412,...,0.329,0.423,0.458,0.313,0.448,0.517,0.293,0.421,0.583,0.347
150,0.527,0.608,0.354,0.541,0.533,0.342,0.543,0.533,0.351,0.543,...,0.331,0.527,0.608,0.354,0.556,0.683,0.394,0.541,0.533,0.342
175,0.561,0.633,0.520,0.536,0.617,0.532,0.550,0.700,0.484,0.536,...,0.502,0.561,0.633,0.520,0.561,0.683,0.550,0.536,0.617,0.532
200,0.640,0.683,0.495,0.599,0.658,0.507,0.635,0.633,0.466,0.626,...,0.468,0.640,0.683,0.495,0.574,0.608,0.464,0.599,0.658,0.507
225,0.538,0.617,0.516,0.493,0.583,0.493,0.543,0.558,0.514,0.493,...,0.493,0.538,0.617,0.516,0.545,0.683,0.525,0.493,0.583,0.493


In [73]:
# seed= '1' # or image
final_score_list = []
seed_list = [1, 3, 4, 5]
for seed in seed_list:
    train_score = all_result[f'petnet_image_train_{seed}'].values
    test_score = all_result[f'petnet_image_test_{seed}'].values
    final_score = get_final_score(train_score, test_score)
    final_score_list.append(final_score)
print(final_score_list)
print('mean', np.around(np.mean(final_score_list), 3), 'std', np.around(np.std(final_score_list), 3))

[0.691, 0.653, 0.687, 0.678]
mean 0.677 std 0.015


# Experiment1.

In [ ]:
experiment_name = 'task_id_1'

f = f'../experiment1/{experiment_name}/eval/*entence_type_language/random_seed_*'
# f = f'../experiment2/{experiment_name}/eval/sentence_type_image/random_seed_*'
fs = natsorted(glob.glob(f))
print(fs)
random_seed = []
for i in fs:
    random_seed.append(i.split('/')[-1].split('_')[2])
# print('experiment_name: ', experiment_name,)
random_seed = [1, 2, 3]
print('random_seed: ', random_seed)
data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])

def result_from_folder(experiment_name, random_seed=[1]):
    for random in random_seed:    
        folder = f'../experiment1/{experiment_name}/eval/*entence_type_language/random_seed_{random}*/evaluated_gifs/*'
        files = natsorted(glob.glob(folder))
#         print(len(files))
        if not files:
            print('NO FILES')
        for file in files:
#             print(file)
            name = file.split('/')[-1]
#             print(name)
            epoch = int(re.sub(r'\D', '', name))
            if 'new' in name:
                _type = 'new_test'
            elif 'train' in name:
                _type = 'train'
            else:
                _type = 'test'
#             print(epoch, f'{experiment_name}_{_type}_{random}')
#             if random == '1':
#                 print(file)
            with open(file + '/result.json', 'r') as f:
                content = json.load(f)
                if content.get('final_suc'):
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('final_suc')
                else:
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('new_final_suc')
result_from_folder(experiment_name, random_seed)
print(data)

# output_data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])
# for t in ['train', 'test', 'new_test']:  
#     list_random = []
#     for random in random_seed:
#         list_random.append(f'{experiment_name}_{t}_{random}')
#     print(list_random)
#     output_data[f'{experiment_name}_{t}_mean'] = data[list_random].mean(1)
#     output_data[f'{experiment_name}_{t}_std'] = data[list_random].std(1)
# output_data

: 

In [10]:
# save to numpyz.
name = experiment_name + '.npz'
np_data = np.array(data)
np_header = np.array(data.columns.tolist())
print(name)
np.savez(name, np_header, np_data)

task_id_1.npz


In [14]:
# load from numpyz.
d = np.load('bc_language.npz', allow_pickle=True)
pd.DataFrame(d['arr_1'], columns=d['arr_0'])

FileNotFoundError: [Errno 2] No such file or directory: 'bc_language.npz'

 Methods & train &  test & new-test \\\hline\hline

PETNet(init image) & 57.8 \% & 36.4 \% & 29.3 \% \\
MILL(init image) & 55.0 \% & 43.0 \% & 37.4 \% \\
\textbf{PETNet(last image)} & \textbf{83.3} \% & \textbf{71.8} \% & \textbf{70.0} \% \\
MILL(last image) & 80.0 \% & 65.1 \% & 63.1 \% \\\hline
PETNet & 73.2\% & 67.6\% & 56.2 \%\\
MILL & 78.3 \% & 69.4 \% & 59.6 \%\\

# Experiment2

In [10]:
experiment_name = 'mill_random_instruction_1_1e-07_last'

f = f'../experiment2/{experiment_name}/eval/*entence_type_both/random_seed_*'
# f = f'../experiment2/{experiment_name}/eval/sentence_type_image/random_seed_*'
fs = natsorted(glob.glob(f))
print(fs)
random_seed = []
for i in fs:
    random_seed.append(i.split('/')[-1].split('_')[2])
# print('experiment_name: ', experiment_name,)
# random_seed = [1, 2, 3]
print('random_seed: ', random_seed)
data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])

def result_from_folder(experiment_name, random_seed=[1]):
    for random in random_seed:    
        folder = f'../experiment2/{experiment_name}/eval/*entence_type_both/random_seed_{random}*/evaluated_gifs/*'
        files = natsorted(glob.glob(folder))
#         print(len(files))
        if not files:
            print('NO FILES')
        for file in files:
#             print(file)
            name = file.split('/')[-1]
#             print(name)
            epoch = int(re.sub(r'\D', '', name))
            if 'new' in name:
                _type = 'new_test'
            elif 'train' in name:
                _type = 'train'
            else:
                _type = 'test'
#             print(epoch, f'{experiment_name}_{_type}_{random}')
#             if random == '1':
#                 print(file)
            with open(file + '/result.json', 'r') as f:
                content = json.load(f)
                if content.get('final_suc'):
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('final_suc')
                else:
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('new_final_suc')
result_from_folder(experiment_name, random_seed)
print(data)

['../experiment2/mill_random_instruction_1_1e-07_last/eval/last_sentence_type_both/random_seed_2_2021-06-15-15-35', '../experiment2/mill_random_instruction_1_1e-07_last/eval/last_sentence_type_both/random_seed_3_2021-06-15-15-35', '../experiment2/mill_random_instruction_1_1e-07_last/eval/sentence_type_both/random_seed_1_2021-06-11-14-00']
random_seed:  ['2', '3', '1']
        mill_random_instruction_1_1e-07_last_test_2  \
25000                                      0.250000   
50000                                      0.288288   
75000                                      0.385135   
100000                                     0.515766   
125000                                     0.581081   
150000                                     0.596847   
175000                                     0.531532   
200000                                     0.650901   
225000                                     0.563063   
250000                                     0.540541   
275000                  

In [12]:
# save to numpyz.
name = experiment_name + '.npz'
np_data = np.array(data)
np_header = np.array(data.columns.tolist())
print(name)
np.savez(name, np_header, np_data)

mill_random_instruction_1_1e-07_last.npz


# Experiment3

## instruction_types
DONE

In [10]:
# experiment_name = 'mill_random_instruction_1_1e-07_last'
experiment_name = 'mill_word_instruction_1_1e-07'
sentence_type = 'language'
f = f'../ablation/1_instruction_types/{experiment_name}/eval/*entence_type_{sentence_type}/random_seed_*'
# f = f'../experiment2/{experiment_name}/eval/sentence_type_image/random_seed_*'
fs = natsorted(glob.glob(f))
print(fs)
random_seed = []
for i in fs:
    random_seed.append(i.split('/')[-1].split('_')[2])
# print('experiment_name: ', experiment_name,)
random_seed = [1]
print('random_seed: ', random_seed)
data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])

def result_from_folder(experiment_name, random_seed=[1]):
    for random in random_seed:    
        folder = f'../ablation/1_instruction_types/{experiment_name}/eval/*entence_type_{sentence_type}/random_seed_{random}_*/evaluated_gifs/*'
        files = natsorted(glob.glob(folder))
#         print(files)
        if not files:
            print('NO FILES')
        for file in files:
#             print(file)
            name = file.split('/')[-1]
#             print(name)
            epoch = int(re.sub(r'\D', '', name))
            if 'new' in name:
                _type = 'new_test'
            elif 'train' in name:
                _type = 'train'
            else:
                _type = 'test'
#             print(epoch, f'{experiment_name}_{_type}_{random}')
#             if random == '1':
#                 print(file)
            with open(file + '/result.json', 'r') as f:
                content = json.load(f)
                if content.get('final_suc'):
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('final_suc')
                else:
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('new_final_suc')
result_from_folder(experiment_name, random_seed)
print(data)

['../ablation/1_instruction_types/mill_word_instruction_1_1e-07/eval/first_last_sentence_type_language/random_seed_1_2021-06-15-21-46']
random_seed:  [1]
        mill_word_instruction_1_1e-07_test_1  \
25000                               0.211712   
50000                               0.274775   
75000                               0.367117   
100000                              0.423423   
125000                              0.382883   
150000                              0.412162   
175000                              0.396396   
200000                              0.416667   
225000                              0.418919   
250000                              0.470721   
275000                              0.434685   
300000                              0.407658   
325000                              0.445946   
350000                              0.443694   
375000                              0.461712   
400000                              0.448198   

        mill_word_instruction

In [12]:
# save to numpyz.
name = experiment_name + '_' + sentence_type +'.npz'
np_data = np.array(data)
np_header = np.array(data.columns.tolist())
print(name)
np.savez(name, np_header, np_data)

mill_word_instruction_1_1e-07_language.npz


## hypa

random_instruction(15types), sentence_type is Language and Both. seed=1.
## training 05, 06, 09, 10.

doing: 01, 02, 03, 04 language, 1.both,language.

_next: 05, 06, 09, 10. language, both. & 0. 1. ?

08. DONE.
07. DONE.

In [59]:
# experiment_name = 'mill_random_instruction_1_1e-07_last'
experiment_name = 'mill_random_instruction_1*'
sentence_type = 'both'
f = f'../ablation/2_hypa/{experiment_name}/eval/*entence_type_{sentence_type}/random_seed_*/evaluated_gifs/*'
fs = natsorted(glob.glob(f))
# print(fs)
random_seed = []
# for i in fs:
#     random_seed.append(i.split('/')[-1].split('_')[2])

random_seed = [1]
print('random_seed: ', random_seed)
data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])

_f = f'../ablation/2_hypa/{experiment_name}/eval/*entence_type_{sentence_type}/random_seed_*/evaluated_gifs/*'
_fs = natsorted(glob.glob(_f))
# print(_fs)
for file in fs:
    name = file.split('/')[-1]
    experiment_name = file.split('/')[3].split('_')[-1]
    epoch = int(re.sub(r'\D', '', name))
    if 'new' in name:
        _type = 'new_test'
    elif 'train' in name:
        _type = 'train'
    else:
        _type = 'test'
    with open(file + '/result.json', 'r') as f:
        content = json.load(f)
        if content.get('final_suc'):
            data.loc[epoch, f'{experiment_name}_{_type}'] = content.get('final_suc')
        else:
            data.loc[epoch, f'{experiment_name}_{_type}'] = content.get('new_final_suc')
#     print(file)
#     print(experiment_name)
    
print(data.columns)

random_seed:  [1]
Index(['1_test', '1_train', '1_new_test', '0.01_test', '0.001_test',
       '0.0001_test', '0.01_train', '0.001_train', '0.0001_train',
       '0.01_new_test', '0.001_new_test', '0.0001_new_test', '0.1_test',
       '0.1_train', '0.1_new_test', '1e-05_test', '1e-05_train',
       '1e-05_new_test', '1e-06_test', '1e-06_train', '1e-06_new_test',
       '1e-07_test', '1e-07_train', '1e-07_new_test', '1e-08_test',
       '1e-08_train', '1e-08_new_test', '1e-09_test', '1e-09_train',
       '1e-09_new_test', '1e-10_test', '1e-10_train', '1e-10_new_test'],
      dtype='object')


In [61]:
# save to numpyz.
name = 'mill_hypa_both.npz'
np_data = np.array(data)
np_header = np.array(data.columns.tolist())
print(name)
np.savez(name, np_header, np_data)

mill_hypa_both.npz


In [58]:
def result_from_folder(experiment_name, random_seed=[1]):
    for random in random_seed:    
        folder = f'../ablation/2_hypa/{experiment_name}/eval/*entence_type_{sentence_type}/random_seed_*/evaluated_gifs/*'
        files = natsorted(glob.glob(folder))
#         print(files)
        if not files:
            print('NO FILES')
        for file in files:
#             print(file)
            name = file.split('/')[-1]
#             print(name)
            epoch = int(re.sub(r'\D', '', name))
            if 'new' in name:
                _type = 'new_test'
            elif 'train' in name:
                _type = 'train'
            else:
                _type = 'test'
#             print(epoch, f'{experiment_name}_{_type}_{random}')
#             if random == '1':
#                 print(file)
            with open(file + '/result.json', 'r') as f:
                content = json.load(f)
                if content.get('final_suc'):
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('final_suc')
                else:
                    data.loc[epoch, f'{experiment_name}_{_type}_{random}'] = content.get('new_final_suc')
result_from_folder(experiment_name, random_seed)
print(data)

NO FILES
          1_test  0.01_test  0.001_test  0.0001_test  0.1_test  1e-05_test  \
25000   0.171171   0.166667    0.313063     0.299550  0.184685    0.144144   
50000   0.180180   0.231982    0.218468     0.283784  0.295045    0.292793   
75000   0.310811   0.234234    0.250000     0.231982  0.391892    0.245495   
100000  0.340090   0.308559    0.340090     0.337838  0.304054    0.261261   
125000  0.306306   0.306306    0.195946     0.270270  0.283784    0.301802   
150000  0.281532   0.333333    0.299550     0.376126  0.324324    0.319820   
175000  0.324324   0.349099    0.268018     0.360360  0.351351    0.286036   
200000  0.353604   0.337838    0.364865     0.373874  0.299550    0.313063   
225000  0.315315   0.315315    0.319820     0.369369  0.340090    0.295045   
250000  0.326577   0.380631    0.340090     0.317568  0.353604    0.355856   
275000  0.362613   0.344595    0.394144     0.414414  0.382883    0.301802   
300000  0.391892   0.364865    0.331081     0.319820  0